#   <center> <span style="color:gray"> Azure Linux VM Size and Price at Glance </span> </center>

##   <span style="color:Gray"> Overview </span>
<span  STYLE="color: black font-size: 12pt"> This notebook was developped to assist Oracle Team to have the VM Sizes and Prices for all available Azure VM SKUs in one place to assist with database worload sizing.  </span>
<p>The key metrics for proper database workload sizing include: vCPUs, memory, Max IOPS, Max IO Throughput MBPS, temporary storage for Swap space, monthly cost, Max Data disks allowed etc. </p>
<p>Typical database workloads on IaaS VM are usually IO bound instead of CPU or Memory at the VM level. It is critical to size the VM based on the database workload leveraging Oracle AWR (Automated Workload Repository). </p>
<p>Azure VM size information can be found in https://docs.microsoft.com/en-us/azure/virtual-machines/sizes. However the information is scattered in multiple linked pages which made it difficult to navigate and compare. VM pricing can be obtained by API calls to https://prices.azure.com/. In our example, we will only obtain the consumption based retail prices for Linux based VMs. </p>
<p>The goal of this notebook is to combine all these information (VM specifications and cost) in one spreadsheet for easy comparison and decision making.  We use BeautifulSoup to scrap the information from Microsoft document sites for VM sizes and specficications. The final product is an excel sheet with key metrics for all available Azure Linux VMs. </p>
<p>Note: The code would work for future released VM size as well as long as the embeded size table in the microsoft VM size page has the same/similar column formats using the same tabs (basics, local storage, remote storage, network). </p>


In [16]:
%pip install -q --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install pandas beautifulsoup4 requests lxml

#%pip install selenium
#%pip install webdriver-manager

%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import urlparse, parse_qs
import re, os, io, zipfile
import json
import requests
from requests import exceptions
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from urllib.parse  import  urljoin
 

In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
def requests_retry_session(retries=3, 
                           backoff_factor=0.3, 
                           status_forcelist=(500, 502, 503, 504), 
                           session=None):
    '''
    Use/Create an http(s) requests session that will retry a request.
    '''
    session = session or requests.Session()
    retry = Retry(total = retries, 
                  read = retries, 
                  connect = retries, 
                  backoff_factor = backoff_factor, 
                  status_forcelist = status_forcelist)
    adapter = HTTPAdapter(max_retries = retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    return session

    


We will use API calls to obtain the Azure Linux VM retail prices for consumption based SKUs (Not reserved or spot) for East US 2.
Save the pricing data as a csv at the end. 

In [21]:

dfpricing = pd.DataFrame()

# Each API call returns up to 1000 records; loop to fetch enough for region "eastus2"
for i in range(0, 50000, 1000):
    try:
        #url = f'https://prices.azure.com/api/retail/prices?$filter=armRegionName%eq%\'eastus2\'&$skip={i}'
        url = f'https://prices.azure.com/api/retail/prices?$filter=armRegionName%20eq%20%27eastus2%27%20and%20serviceName%20eq%20%27Virtual%20Machines%27&$skip={i}'
        df = pd.read_json(url)

        if df.empty:
            print(f"Empty at skip={i}")
            break

        df = pd.json_normalize(df['Items'])

        # Filter for Linux Virtual Machines
        #df_vm_notspot = df[
        #    ~df['skuName'].str.contains("spot", case=False, na=False)
        #]
        df_vm_notspot = df[
             ~df['skuName'].str.contains("spot", case=False, na=False) &
             ~df['skuName'].str.contains("low priority", case=False, na=False)
        ]
        df_vm_consumption = df_vm_notspot[
            df_vm_notspot['type'].str.startswith("Consumption", na=False)
        ]
        df_vm_linux = df_vm_consumption[
            ~df_vm_consumption['productName'].str.contains("Windows", case=False, na=False)
        ]
 
 
        dfpricing = pd.concat([dfpricing, df_vm_linux], ignore_index=True)

    except Exception as e:
        print(f"Error at skip={i}: {e}")
        break

dfpricing.to_csv('azure_linux_vm_prices.csv', index=False)
print(f"Saved {len(dfpricing)} Linux VM pricing entries.")


                

Empty at skip=12000
Saved 1320 Linux VM pricing entries.


In [22]:
dfpricing.head(10)

,currencyCode,tierMinimumUnits,retailPrice,unitPrice,armRegionName,location,effectiveStartDate,meterId,meterName,productId,...,skuName,serviceName,serviceId,serviceFamily,unitOfMeasure,type,isPrimaryMeterRegion,armSkuName,reservationTerm,effectiveEndDate
0,USD,0.0,7.346,7.346,eastus2,US East 2,2022-06-01T00:00:00Z,00230cfc-461f-54a1-80ac-cdacb8293e3c,NC48ads_A100_v4,DZH318Z09TGJ,...,Standard_NC48ads_A100_v4,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_NC48ads_A100_v4,NaN,NaN
1,USD,0.0,2.304,2.304,eastus2,US East 2,2021-12-01T00:00:00Z,29c77562-9c61-56fd-b1cf-747dda4d809d,E32-8ds v5,DZH318Z096SN,...,E32-8ds v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,False,Standard_E32-8ds_v5,NaN,NaN
2,USD,0.0,4.234,4.234,eastus2,US East 2,2024-10-01T00:00:00Z,00769fa9-482e-5871-9b66-1ab22143280c,E64-32s v6,DZH318Z0GNH6,...,E64-32s v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_E64-32s_v6,NaN,NaN
3,USD,0.0,3.616,3.616,eastus2,US East 2,2023-08-01T00:00:00Z,00ba6c77-0884-59bb-b707-776b88b32e25,DC64edsv5,DZH318Z0K804,...,DC64edsv5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_DC64eds_v5,NaN,NaN
4,USD,0.0,3.348,3.348,eastus2,US East 2,2023-07-01T00:00:00Z,01b30754-7a73-5fb7-9f57-3f1a750c0b87,FX36mds,DZH318Z08Q75,...,Standard_FX36mds,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_FX36mds,NaN,NaN
5,USD,0.0,2.976,2.976,eastus2,US East 2,2024-10-01T00:00:00Z,02315619-d8e9-5d9e-ad5e-ee048252d031,D96pls v6,DZH318Z0G1GP,...,D96pls v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D96pls_v6,NaN,NaN
6,USD,0.0,6.288,6.288,eastus2,US East 2,2021-11-01T00:00:00Z,023f9cfd-1088-5b37-a020-3290d0c5bc62,E96ads v5,DZH318Z093X0,...,E96ads v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_E96ads_v5,NaN,NaN
7,USD,0.0,3.616,3.616,eastus2,US East 2,2021-11-01T00:00:00Z,0287cb43-aa9a-5fae-89da-27bfc042827b,E64-16as v5,DZH318Z093WZ,...,E64-16as v5,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_E64-16as_v5,NaN,NaN
8,USD,0.0,2.312,2.312,eastus2,US East 2,2024-08-01T00:00:00Z,02a24596-f4f2-58f3-a8df-a477e2f2f215,D32nlds v6,DZH318Z0LDQX,...,D32nlds v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_D32nlds_v6,NaN,NaN
9,USD,0.0,0.546,0.546,eastus2,US East 2,2024-09-01T00:00:00Z,02a7e7c4-71c2-5808-9897-168f8a252365,F8as v6,DZH318Z0FXXJ,...,F8as v6,Virtual Machines,DZH313Z7MMC8,Compute,1 Hour,Consumption,True,Standard_F8as_v6,NaN,NaN


##  
<span  STYLE="color: black font-size: 15pt"> Below is to parse and format the json file we scrap from the embedded VM size table. </span>

In [23]:
# THis is for older SKUs that don't have multiple tabs (e.g., basics, local storage, remote storage, network etc.)
# URL of the page
# We may not need this anymore, but keeping it for reference
url = "https://learn.microsoft.com/en-us/azure/virtual-machines/ev3-esv3-series?toc=%2Fazure%2Fvirtual-machines%2Flinux%2Ftoc.json&bc=%2Fazure%2Fvirtual-machines%2Flinux%2Fbreadcrumb%2Ftoc.json"

# Send a GET request
response = requests.get(url)
response.raise_for_status()  # Raise an error for bad status

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables
tables = soup.find_all('table')

# Convert the first two tables to DataFrames
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]

# Display the DataFrames
print("EV3 Series Table:")
print(df1)
print("\nESV3 Series Table:")
print(df2)


EV3 Series Table:
                 Size  vCPU  Memory: GiB  Temp storage (SSD) GiB  \
0     Standard_E2_v31     2           16                      50   
1      Standard_E4_v3     4           32                     100   
2      Standard_E8_v3     8           64                     200   
3     Standard_E16_v3    16          128                     400   
4     Standard_E20_v3    20          160                     500   
5     Standard_E32_v3    32          256                     800   
6     Standard_E48_v3    48          384                    1200   
7     Standard_E64_v3    64          432                    1600   
8  Standard_E64i_v3 2    64          432                    1600   

   Max data disks Max temp storage throughput: IOPS / Read MBps / Write MBps  \
0               4                                         3000/46/23           
1               8                                         6000/93/46           
2              16                                       12000

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


# Function to clean column names
# This function will remove superscripts, subscripts, and digits from column names

def clean_column_names(df):
    # Custom mapping dictionary. normalize the column names to a standard format
    column_mapping = {
        "Max Temp Storage (Qty.)": "Max Temp Storage Disks (Qty.)",
        "Temp Storage Size (GiB)": "Temp Disk Size (GiB)",
        "Temp Disk Random Read (RR) IOPS": "Temp ReadOnly Disk IOPS",
        "Temp Disk Random Read (RR) Throughput (MB/s)": "Temp ReadOnly Disk Speed (MBps)",
        "Temp Disk Random Write (RW) IOPS": "Temp ReadWrite Disk IOPS",
        "Temp Disk Random Write (RW) Throughput (MB/s)": "Temp ReadWrite Disk Speed (MBps)",
        "Max Remote Storage (Qty.)": "Max Remote Storage Disks (Qty.)",
        "Temp ReadWrite Storage IOPS": "Temp ReadWrite Disk IOPS",
        "Temp ReadWrite Storage Speed (MBps)": "Temp ReadWrite Disk Speed (MBps)",
        "Uncached Premium SSD Disk IOPS": "Uncached Storage IOPS",
        "Uncached Premium SSD Throughput (MB/s)": "Uncached Storage Speed (MBps)",
        "Memory (GB)": "Memory (GiB)",
        "Uncached Premium SSD Burst IOPS": "Uncached Storage Burst IOPS",
        "Uncached Premium SSD Burst Throughput (MB/s)": "Uncached Storage Burst Speed (MBps)",
    }

    # Clean each column name
    cleaned_columns = []
    for col in df.columns:
        # Remove superscripts, subscripts, and digits
        col = re.sub(r'[\u00B9\u00B2\u00B3\u2070-\u209F\d]', '', col)
        # Normalize whitespace: remove leading/trailing and reduce multiple spaces to one
        col = re.sub(r'\s+', ' ', col).strip()
        cleaned_columns.append(col)

    # Apply mapping
    df.columns = [column_mapping.get(col, col) for col in cleaned_columns]

    return df

# Function to parse VM specs from the given URL extracting all information from the tabs (basics, local storage, remote storage, network etc.)
def parse_vm_specs(url):
     
    #url = "https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/edsv4-series"

    # Fetch the page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Tab sections and friendly labels
    # leave out "accelerator" since it's not relevant for this (GPU etc)
    tabs = {
        "tabpanel_1_sizebasic": "Basics",
        "tabpanel_1_sizestoragelocal": "Local Storage",
        "tabpanel_1_sizestorageremote": "Remote Storage",
        "tabpanel_1_sizenetwork": "Network"
    }



    dataframes = {}

    for section_id, label in tabs.items():
        section = soup.find("section", {"id": section_id})
        if section:
            table = section.find("table")
            if table:
                df = pd.read_html(str(table))[0]
                #df["Tab"] = label  # Optional: add tab label as a column
                dataframes[label] = df
            else:
                print(f"No table found in section: {label}")
        else:
            print(f"Section not found: {section_id}")

    # Access each dataframe like this:
    df_basic = dataframes.get("Basics")
    df_local = dataframes.get("Local Storage")
    df_remote = dataframes.get("Remote Storage")
    df_network = dataframes.get("Network")
    

    for key in dataframes:
        dataframes[key] = clean_column_names(dataframes[key])

 


    # Step 1: Set "Size Name" as index for each DataFrame
    


    for key in dataframes:
        if "Size Name" in dataframes[key].columns:
            dataframes[key] = dataframes[key].set_index("Size Name")

    # Ensure all expected tabs exist, even if empty
    for label in ["Basics", "Local Storage", "Remote Storage", "Network"]:
        if label not in dataframes:
            dataframes[label] = pd.DataFrame()

    # Step 2: Merge all DataFrames on "Size Name"
    # Start with the Basics DataFrame (must exist)
    df_merged = dataframes["Basics"]

    for label in ["Local Storage", "Remote Storage", "Network"]:
        if not dataframes[label].empty:
            df_merged = df_merged.join(dataframes[label], how="outer", rsuffix=f" ({label})")
        else:
            print(f"Tab '{label}' missing or empty. Columns will be null.")

    # Step 3: Reset index to get "Size Name" back as a column
    df_merged.reset_index(inplace=True)

    return df_merged

# Example usage     
url = "https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/edsv4-series"
df_vm_sizes = parse_vm_specs(url)
df_vm_sizes.to_csv("azure_vm_sizes_memory_optimized.csv", index=False)
print("Saved VM sizes to azure_vm_sizes_memory_optimized.csv")  



Saved VM sizes to azure_vm_sizes_memory_optimized.csv


In [25]:
 

# Function to get all links from the page each VM sku family. The hrefs are relative, so we need to add the base URL. 

def get_vmsize_links(url,base_url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all <a> tags with "View the" in the text
    links = soup.find_all("a", string=lambda text: text and "View the full" in text)

    # Extract the hrefs and build absolute URLs
    hrefs = [base_url + link.get("href") for link in links if link.get("href")]

    # Optional: Print all found URLs
    for href in hrefs:
        print(href)
    return hrefs

# for Oracle workloads, we usually only need the E-series and D-series
hrefs_memory = get_vmsize_links("https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/e-family","https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/")
hrefs_general=get_vmsize_links("https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/general-purpose/d-family","https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/general-purpose/")
hrefs=hrefs_memory + hrefs_general

# Loop through each link and parse the VM specs
df_all = pd.DataFrame()
for url in hrefs:
    try:
        df_vm_sizes = parse_vm_specs(url)
        df_all = pd.concat([df_all, df_vm_sizes], ignore_index=True)
    except Exception as e:
        print(f"Error processing {url}: {e}")

https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/epsv6-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/epdsv6-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/easv6-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/eadsv6-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/ev5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/esv5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/edv5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/edsv5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/easv5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-optimized/eadsv5-series
https://learn.microsoft.com/en-us/azure/virtual-machines/sizes/memory-o

In [26]:
df_all.reset_index(drop=True, inplace=True)
df_all.to_csv("azure_vm_sizes_all.csv", index=False)

In [27]:
# Merge with pricing data
dfpricing = dfpricing[['armSkuName', 'retailPrice', 'unitOfMeasure','location']]
dfpricing.rename(columns={'retailPrice': 'Price/hour', 'unitOfMeasure': 'Unit'}, inplace=True)
df_all_with_prices = pd.merge(df_all, dfpricing, how='left', left_on='Size Name', right_on='armSkuName')
df_all_with_prices = df_all_with_prices.drop(columns=['armSkuName'])
df_all_with_prices.to_csv("azure_vm_sizes_all_with_prices.csv", index=False)

In [29]:
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

# File path
excel_path = 'e_d_vm_sizes_with_prices.xlsx'

# Write to Excel
df_all_with_prices.to_excel(excel_path, index=False)



# Load workbook and worksheet
wb = load_workbook(excel_path)
ws = wb.active

# Define styles
header_font = Font(bold=True)
wrap_alignment = Alignment(wrap_text=True)
fill1 = PatternFill(start_color="DCE6F1", end_color="DCE6F1", fill_type="solid")  # Light Blue 1
fill2 = PatternFill(start_color="EBF1DE", end_color="EBF1DE", fill_type="solid")  # Light Blue 2

# Style header row
for col, cell in enumerate(ws[1], 1):
    cell.font = header_font
    cell.alignment = wrap_alignment

    # Auto-size columns based on header length
    column_letter = get_column_letter(col)
    #max_length = len(str(cell.value))
    max_length = 1
    
    # Also check data rows for sizing
    for row in range(2, ws.max_row + 1):
        value = ws.cell(row=row, column=col).value
        if value:
            max_length = max(max_length, len(str(value)))
    
    ws.column_dimensions[column_letter].width = max_length + 4  # Add padding

# Apply alternating fill colors to rows
for row in range(2, ws.max_row + 1):
    fill = fill1 if row % 2 == 0 else fill2
    for cell in ws[row]:
        cell.fill = fill

# Save changes
wb.save(excel_path)
